## Árvores de regressão - exercícios 02

Este exercício será uma continuação do anterior, mesma base, mesmas variáveis - vamos tentar buscar a 'melhor árvore'.


*Atenção - Utilizar a base de dados em anexo que é a mesma base que utilizamos na atividade anterior! A base Boston, assim como para a primeira atividade foi descontinuada e não deve ser utilizada*

In [ ]:
import pandas as pd

import seaborn as sns

from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split

### 1. Execute os passos do exercício anterior, até que você tenha uma árvore de regressão predizendo o valor do imóvel na base de treinamento.

In [ ]:
from sklearn.metrics import mean_squared_error

df = pd.read_csv('housing.csv')

df = df.dropna()

df = pd.get_dummies(df, columns=['ocean_proximity'], drop_first=True)

X = df.drop('median_house_value', axis=1)
y = df['median_house_value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

tree_8 = DecisionTreeRegressor(max_depth=8, random_state=42)
tree_8.fit(X_train, y_train)

tree_2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_2.fit(X_train, y_train)

y_pred_train_8 = tree_8.predict(X_train)
y_pred_test_8 = tree_8.predict(X_test)

y_pred_train_2 = tree_2.predict(X_train)
y_pred_test_2 = tree_2.predict(X_test)

mse_train_8 = mean_squared_error(y_train, y_pred_train_8)
mse_test_8 = mean_squared_error(y_test, y_pred_test_8)

mse_train_2 = mean_squared_error(y_train, y_pred_train_2)
mse_test_2 = mean_squared_error(y_test, y_pred_test_2)

print(f"MSE Árvore Profundidade 8 - Treinamento: {mse_train_8:.2f}")
print(f"MSE Árvore Profundidade 8 - Teste: {mse_test_8:.2f}")
print(f"MSE Árvore Profundidade 2 - Treinamento: {mse_train_2:.2f}")
print(f"MSE Árvore Profundidade 2 - Teste: {mse_test_2:.2f}")

### 2.  Calcule o caminho indicado pelos CCP-alfas dessa árvore.

In [ ]:
import matplotlib.pyplot as plt

tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)

path = tree.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas
impurities = path.impurities

plt.figure(figsize=(10, 5))
plt.plot(ccp_alphas, impurities, marker='o', drawstyle="steps-post")
plt.title("Caminho de poda de complexidade (CCP Alpha)")
plt.xlabel("ccp_alpha")
plt.ylabel("Impureza Total da Árvore")
plt.grid(True)
plt.show()

### 3. Paca cada valor de alpha obtido no item 2, treine uma árvore com o respectivo alfa, e guarde essa árvore em uma lista.

In [ ]:
trees = []

for ccp_alpha in ccp_alphas:
    model = DecisionTreeRegressor(random_state=42, ccp_alpha=ccp_alpha)
    model.fit(X_train, y_train)
    trees.append(model)

print(trees)

### 4. Para cada árvore na lista, calcule o MSE da árvore.

In [ ]:
from sklearn.metrics import mean_squared_error

mse_treino = []
mse_teste = []

for tree_model in trees:
    y_train_pred = tree_model.predict(X_train)
    y_test_pred = tree_model.predict(X_test)
    
    mse_treino.append(mean_squared_error(y_train, y_train_pred))
    mse_teste.append(mean_squared_error(y_test, y_test_pred))

for i in range(len(trees)):
    print(f"ccp_alpha: {ccp_alphas[i]:.5f} | MSE Treino: {mse_treino[i]:.2f} | MSE Teste: {mse_teste[i]:.2f}")

### 5. Monte um gráfico do MSE pelo alpha, escolha um valor de alpha perto do ponto de mínimo do MSE

In [ ]:

plt.figure(figsize=(10, 6))

# Plotando o MSE de treino
plt.plot(ccp_alphas, mse_treino, marker='o', label='MSE Treinamento', color='blue')

# Plotando o MSE de teste
plt.plot(ccp_alphas, mse_teste, marker='o', label='MSE Teste', color='red')

plt.xlabel("ccp_alpha")
plt.ylabel("Erro Quadrático Médio (MSE)")
plt.title("MSE em função do ccp_alpha")
plt.legend()
plt.grid(True)
plt.show()


### 6. Calcule o R-quadrado dessa árvore encontrada no item acima

In [ ]:

i = mse_teste.index(min(mse_teste))

melhor_arvore = arvores_ccp[i]

r2_treino = melhor_arvore.score(X_train, y_train)

r2_teste = melhor_arvore.score(X_test, y_test)

print(f"R² (treinamento): {r2_treino:.4f}")
print(f"R² (teste): {r2_teste:.4f}")


### 7. Visualize esta árvore.